In [1]:
# Astronomy 400B
# Homework 4
# Center of Mass Position and Velocity
# Marco Barragan
# Note: 
#    Template Used

In [2]:
# Import modules
import numpy as np
import astropy.units as u
import astropy.table as tbl
from ReadFile import Read

In [3]:
class CenterOfMass:
# Class to define COM position and velocity properties of a given galaxy and simulation snapshot
    
    
    def __init__(self, filename, ptype):
    # Initialize the instance of this Class with the following properties:
    
        # Read data in the given file using Read
        self.time, self.total, self.data = Read(filename)                                                                                             

        # Create an array to store indexes of particles of desired Ptype                                
        self.index = np.where(self.data['type'] == ptype)

        # Store the mass, positions, velocities of only the particles of the given type
        self.m = self.data['m'][self.index]
        self.x = self.data['x'][self.index]
        self.y = self.data['y'][self.index]
        self.z = self.data['z'][self.index]
        self.vx = self.data['vx'][self.index]
        self.vy = self.data['vy'][self.index]
        self.vz = self.data['vz'][self.index]


    def COMdefine(self,a,b,c,m):
    # Function to compute the center of mass position or velocity generically
    # Input: 
    #    array (a,b,c) of positions or velocities
    #    m is the mass
    # Returns: 
    #    3 floats (the center of mass coordinates)

        # x-component Center of mass
        Acom = np.sum(a*m) / np.sum(m)
        # y-component Center of mass
        Bcom = np.sum(b*m) / np.sum(m)
        # z-component Center of mass
        Ccom = np.sum(c*m) / np.sum(m)
        
        return Acom, Bcom, Ccom
    
    
    def COM_P(self, delta):
    # Function to specifically return the center of mass position and velocity                                         
    # Input:                                                                                                           
    #    particle type (1,2,3)                                                                                     
    #    delta (tolerance)                                                                                         
    # Returns: One vector, with rows indicating:                                                                                                                                                                            
    #    COM is the set of 3D coordinates of the center of mass position (kpc)                                                             

        # Center of Mass Position                                                                                      
        ###########################                                                                                    

        # Try a first guess at the COM position by calling COMdefine                                                   
        XCOM, YCOM, ZCOM = self.COMdefine(self.x, self.y, self.z, self.m)

        # compute the magnitude of the COM position vector.
        RCOM = np.sqrt(XCOM**2 + YCOM**2 + ZCOM**2)


        # Iterative process to determine the center of mass                                                            

        # Change reference frame to COM frame                                                                          
        # Compute the difference between particle coordinates and the first guess at COM position
        xNew = self.x - XCOM
        yNew = self.y - YCOM
        zNew = self.z - ZCOM
        RNEW = np.sqrt(xNew**2 + yNew**2 + zNew**2)

        # Find the max 3D distance of all particles from the guessed COM                                               
        # Will re-start at half that radius (reduced radius)                                                           
        RMAX = max(RNEW)/2.0
        
        # Pick an initial value for the change in COM position                                                      
        # between the first guess above and the new one computed from half that volume
        # It should be larger than the input tolerance (delta) initially
        CHANGE = 1000.0

        # Start iterative process to determine center of mass position                                                 
        # "delta" is the tolerance for the difference in the old COM and the new one.    
        while (CHANGE > delta):
            # Select all particles within the reduced radius (starting from original x,y,z, m)
            index2 = np.where(RNEW<RMAX)
            x2 = self.x[index2]
            y2 = self.y[index2]
            z2 = self.z[index2]
            m2 = self.m[index2]

            # Refined COM position:                                                                                    
            # Compute the center of mass position using the particles in the reduced radius
            XCOM2, YCOM2, ZCOM2 = self.COMdefine(x2, y2, z2, m2)
            
            # Compute the new 3D COM position
            RCOM2 = np.sqrt(XCOM2**2 + YCOM2**2 + ZCOM2**2)

            # Determine the difference between the previous center of mass position                                    
            # and the new one.                                                                                         
            CHANGE = np.abs(RCOM - RCOM2)
            
            # Before loop continues, reset : RMAX, particle separations and COM                                        

            # Reduce the volume by a factor of 2 AGAIN                                                                 
            RMAX = RMAX/2.0                                                                             

            # Change the frame of reference to the newly computed COM.                                                 
            # Subtract the new COM
            xNew = x2 - XCOM2
            yNew = y2 - YCOM2
            zNew = z2 - ZCOM2
            RNEW = np.sqrt(xNew**2 + yNew**2 + zNew**2)

            # Set the center of mass positions to the refined values                                                   
            XCOM = XCOM2
            YCOM = YCOM2
            ZCOM = ZCOM2
            RCOM = RCOM2

            # Create a vector to store the COM position                                                                                                                                                       
            COMP = [XCOM, YCOM, ZCOM]

            
        # Set the correct units usint astropy and round all values
        COM = np.around(COMP,2)*u.kpc
        
        # Return the COM positon vector
        return COM
    

    def COM_V(self, COMX,COMY,COMZ):
        # Center of Mass velocity
        # Input: 
        #    COMX,COMY,COMZ  are the X, Y, Z positions of the COM
        # Returns: 
        #    COMV is the 3D Vector of COM Velocities
        
        # The max distance from the center that we will use to determine the center of mass velocity                   
        RVMAX = 15.0*u.kpc

        # Determine the position of all particles relative to the center of mass position
        xV = self.x*u.kpc - COMX
        yV = self.y*u.kpc - COMY
        zV = self.z*u.kpc - COMZ
        RV = np.sqrt(xV**2 + yV**2 + zV**2)
        
        # Determine the index for those particles within the max radius
        indexV = np.where(RV<RVMAX)

        # Determine the velocity and mass of those particles within the max radius
        vxnew = self.vx[indexV]
        vynew = self.vy[indexV]
        vznew = self.vz[indexV]
        mnew =  self.m[indexV]
        
        # Compute the center of mass velocity using those particles
        VXCOM, VYCOM, VZCOM = self.COMdefine(vxnew, vynew, vznew, mnew)

        # Create a vector to store the COM velocity
        COMV = np.around([VXCOM,VYCOM,VZCOM],2)*u.km/u.s

        # Return the COM vector                                                                                        
        return COMV

In [4]:
# Create a Center of mass object for the MW, M31 and M33
MWCOM = CenterOfMass("MW_000.txt", 2)
M31COM = CenterOfMass("M31_000.txt", 2)
M33COM = CenterOfMass("M33_000.txt", 2)

In [5]:
# Store the position and velocity COM for MW
MW_COMP = MWCOM.COM_P(0.1)
MW_COMV = MWCOM.COM_V(MW_COMP[0], MW_COMP[1], MW_COMP[2])

# Store the position and velocity COM for 31
M31_COMP = M31COM.COM_P(0.1)
M31_COMV = M31COM.COM_V(M31_COMP[0], M31_COMP[1], M31_COMP[2])

# Store the position and velocity COM for M33
M33_COMP = M33COM.COM_P(0.1)
M33_COMV = M33COM.COM_V(M33_COMP[0], M33_COMP[1], M33_COMP[2])

# Questions

1.) What is the COM position and velocity vector for the MW, M31, and M33 at Snapshot 0 using Disk Particles only (ptype = 2) (use 0.1 kpc as the tolerance in order to compare answers)? Recall that the MW COM should be close to the origin of the corrdinate system

In [6]:
# Make a list of the object names
name = ['MW','M31','M33']

# Store the position and velocity COM with respect to their directions
x = [MW_COMP[0].value,M31_COMP[0].value,M33_COMP[0].value] 
y = [MW_COMP[1].value,M31_COMP[1].value,M33_COMP[1].value] 
z = [MW_COMP[2].value,M31_COMP[2].value,M33_COMP[2].value] 

vx = [MW_COMV[0].value,M31_COMV[0].value,M33_COMV[0].value] 
vy = [MW_COMV[1].value,M31_COMV[1].value,M33_COMV[1].value] 
vz = [MW_COMV[2].value,M31_COMV[2].value,M33_COMV[2].value] 


# Make a table of the velocity and COM vectors for each object for disk particles
t = tbl.Table([name,x,y,z,vx,vy,vz], names=('Galaxy Name','COMx (kpc)','COMy (kpc)','COMz (kpc)','COMVx (km/s)','COMVy (km/s)','COMVz (km/s)'))
t.show_in_notebook()

idx,Galaxy Name,COMx (kpc),COMy (kpc),COMz (kpc),COMVx (km/s),COMVy (km/s),COMVz (km/s)
0,MW,-0.87,2.39,-1.42,-0.47,3.41,-1.33
1,M31,-377.66,611.43,-284.64,72.85,-72.14,49.0
2,M33,-476.22,491.44,-412.4,44.42,101.78,142.23


2.) What is the magnitude of the current separation and velocity betwen the MW and M31?

In [7]:
# Find the difference in the COMP and COMV between MW and M31
DeltaCOMP2 = MW_COMP - M31_COMP
MagDeltaCOMP2 = np.sqrt(DeltaCOMP2[0]**2 + DeltaCOMP2[1]**2 + DeltaCOMP2[2]**2)
print(np.around(MagDeltaCOMP2,3))

DeltaCOMV2 = MW_COMV - M31_COMV
MagDeltaCOMV2 = np.sqrt(DeltaCOMV2[0]**2 + DeltaCOMV2[1]**2 + DeltaCOMV2[2]**2)
print(np.around(MagDeltaCOMV2,3))

770.139 kpc
116.691 km / s


3.) What is the magnitude of the current separation and velocity between M33 and M31?

In [8]:
# Find the difference in the COMP and COMV between M33 and M31
DeltaCOMP3 = M33_COMP - M31_COMP
MagDeltaCOMP3 = np.sqrt(DeltaCOMP3[0]**2 + DeltaCOMP3[1]**2 + DeltaCOMP3[2]**2)
print(np.around(MagDeltaCOMP3,3))

DeltaCOMV3 = M33_COMV - M31_COMV
MagDeltaCOMV3 = np.sqrt(DeltaCOMV3[0]**2 + DeltaCOMV3[1]**2 + DeltaCOMV3[2]**2)
print(np.around(MagDeltaCOMV3,3))

201.083 kpc
199.37 km / s


4.) Given that M31 and MW are about to merge, why is the iterative process to determine the COM so important?

Since both objects are about to merge, the outer layers of matter (halo) will begin to interact, thus perturbing the center of mass. Iterating over radius (shortening the radius) allows us to get a better picture of the trajectories of the dynamical system. The halo will be doing all sorts of wierd fluid dynamical things, but the disks and bulges are what we want to see (Because that is mostly what we can see).